In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ==============================
# SEED 설정 (재현성 확보)
# ==============================
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# Early Stopping 클래스
# ==============================
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ==============================
# ResNet 기반 Custom 모델 (추가 CNN 없이)
# ==============================
class CustomResNet(nn.Module):
    def __init__(self, num_classes=6, freeze_backbone=False):
        super(CustomResNet, self).__init__()
        self.backbone = models.resnet50(pretrained=True)  # ResNet-50 사용

        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # 기존 Fully Connected 제거

        # 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)  # ResNet Feature Extraction
        x = x.view(x.size(0), -1)  # Flatten (1D 변환)
        x = self.classifier(x)  # Fully Connected Layer
        return x

# ==============================
# 하이퍼파라미터 설정
# ==============================
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==============================
# 데이터 전처리 & 로더
# ==============================
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet은 224x224 입력 크기 필요
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================
# 모델 및 최적화기 설정
# ==============================
model = CustomResNet(num_classes=num_classes, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ==============================
# 학습 및 평가 함수
# ==============================
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, pred = outputs.max(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, pred = outputs.max(1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    return running_loss/total, correct/total

# ==============================
# 학습 루프
# ==============================
best_test_acc = 0.0
best_epoch = 0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch + 1

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}  | Test Acc: {test_acc:.4f}")

print(f"\nTraining complete! Best Test Accuracy: {best_test_acc:.4f} at Epoch {best_epoch}")


c:\Users\IIALAB\anaconda3\envs\solar\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\IIALAB\anaconda3\envs\solar\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1/50] Train Loss: 1.3828 | Train Acc: 0.5325
  Test Loss: 19.6706  | Test Acc: 0.3446
[Epoch 2/50] Train Loss: 1.1834 | Train Acc: 0.5777
  Test Loss: 1.0934  | Test Acc: 0.6271
[Epoch 3/50] Train Loss: 0.9370 | Train Acc: 0.7090
  Test Loss: 1.1824  | Test Acc: 0.5819
[Epoch 4/50] Train Loss: 0.7807 | Train Acc: 0.7613
  Test Loss: 1.3403  | Test Acc: 0.6723
[Epoch 5/50] Train Loss: 0.7722 | Train Acc: 0.7726
  Test Loss: 1.3671  | Test Acc: 0.6667
[Epoch 6/50] Train Loss: 0.7446 | Train Acc: 0.7627
  Test Loss: 0.9097  | Test Acc: 0.7175
[Epoch 7/50] Train Loss: 0.5943 | Train Acc: 0.8249
  Test Loss: 1.3063  | Test Acc: 0.6949
[Epoch 8/50] Train Loss: 0.6343 | Train Acc: 0.8079
  Test Loss: 1.0158  | Test Acc: 0.7006
[Epoch 9/50] Train Loss: 0.4984 | Train Acc: 0.8460
  Test Loss: 0.8082  | Test Acc: 0.7514
[Epoch 10/50] Train Loss: 0.4562 | Train Acc: 0.8517
  Test Loss: 1.0824  | Test Acc: 0.7006
[Epoch 11/50] Train Loss: 0.4238 | Train Acc: 0.8715
  Test Loss: 0.8014  | Te